<H1>The Transportation Problem</H1>
* Consider a set of warehouses each with a given inventory of widgets, and a set of demand centers each with a given demand for widgets. 
* How many widgets should we ship from each warehouse to each demand center such that all demand is satisfied and shipping costs are minimized?

![Transportation Problem](transportation_problem.png)
<H3> Sets and Indices </H3>
* $i \in I$: Warehouses
* $j \in J$: Customers (demand centers)

<H3>Data</H3>
* $u_i$: capacity for warehouse $i$ (widgets)
* $d_j$: demand at demand center $j$ (widgets)
* $c_{ij}$: shipping cost from warehouse $i$ to customer $j$ ($/widget)

<H3>Decision Variables</H3>
* $x_{ij}$: number of widgets to ship from warehouse $i$ to customer $j$

<H3>Linear Programming Formulation</H3>
\begin{eqnarray}
\min_{x} && \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij} \;\; \mbox{(minimize shipping costs)} \nonumber \\
\mbox{s.t.} && \sum_{i \in I} x_{ij} = d_j,\;\;j \in J \;\; \mbox{(satisfy demand)}\nonumber \\
&& \sum_{j \in J} x_{ij} \le u_i,\;\;i \in I \;\; \mbox{(don't exceed capacity)} \nonumber \\
&& x_{ij} \ge 0, \;\;i \in I,\;j \in J \;\; \mbox{(ship nonnegative quantities)} \nonumber
\end{eqnarray}

<H3>Inputs</H3>

We'll consider a test data set with four demand centers that have the following demands:

In [ ]:
demands = [15, 18, 14, 20]

And with five warehouses that have the following widget capacities:

In [ ]:
capacities = [20, 22, 17, 19, 18]

Finally, we'll need to know the per unit shipping costs between each warehouse and demand center.

In [ ]:
ship_costs =  [[4000, 2500, 1200, 2200],
               [2000, 2600, 1800, 2600],
               [3000, 3400, 2600, 3100],
               [2500, 3000, 4100, 3700],
               [4500, 4000, 3000, 3200]]

<H3>Decision Variables</H3>

For each warehouse, customer pair, we'll need to decide how many units to ship. After adding those variables, we can think about how to generate the linear expressions needed to create the model.

In [ ]:
def get_ship_vars(model, warehouses, customers):
    return {(warehouse, customer): model.addVar(name='ship.w' + str(warehouse) + '.c' + str(customer))
                                   for customer in customers
                                   for warehouse in warehouses}

<H3>Linear Expressions</H3>

This transportation model has two families of constraints. It is important to understand for each constraint which index set is being looped over, and which is being summed over. For demands, there exists one constraint per customer and each constraint involves a sum over the warehouses, and for supplies there exists one constraint per warehouse and each constraint involves a sum over the customers.

Let's create a test model and experiment with generating expressions out of the ship variables.

In [ ]:
import gurobipy as grb
m = grb.Model()
warehouses = range(len(capacities))
customers = range(len(demands))
to_ship = get_ship_vars(m, warehouses, customers)
m.update()
to_ship

We'll pick a customer first, say customer 0, and generate an expression for the number of units that customer received.

In [ ]:
grb.quicksum(to_ship[warehouse, 0] for warehouse in warehouses)

This fixes customer $c_0$, and sums over the warehouses $w_0,\ldots,w_4$. If this works for customer $c_0$, it should work for the rest of the customers if we add an outer loop.

In [ ]:
for customer in customers:
    print grb.quicksum(to_ship[warehouse, customer] for warehouse in warehouses)

This should have generated a linear expression for each customer $c_0,\ldots,c_3$. If so, these are the LinExpr objects we need to construct demand constraints. The takeaway is that the set we are summing over should participate in the inner loop, and the set we are writing the constraint over should participate in the outer loop.

To generate expressions for the supply constraints, we'll need to fix a warehouse and sum over the customers, so we'll want customers on the inner loop and warehouses on the outer.

In [ ]:
for warehouse in warehouses:
    print grb.quicksum(to_ship[warehouse, customer] for customer in customers)

Each LinExpr object we just generated should consider a single warehouse and sum over all customers.

<H3>Constraints</H3>

Generating the correct LinExpr objects for each demand and supply is most of the challenge. We can turn each LinExpr into a constraint by using the overloaded $<=$, $>=$, and $==$ operators.

For the demand constraints, we'll loop over the customers and sum over the warehouses.

Note: We are using Python's list comprehension syntax here, which puts the outer for loop inside the []. It is still the case that the outer loop iterates over the customer, and the inner loop sums over the warehouses.

In [ ]:
def get_demand_constrs(model, to_ship, demands):
    return [model.addConstr(grb.quicksum(to_ship[warehouse, customer] for warehouse in warehouses) == demand,
                            name='demand.' + str(customer))
           for customer, demand in enumerate(demands)]

We can write a similar method to generate the capacity constraints.

In [ ]:
def get_capacity_constrs(model, to_ship, capacities):
    return [model.addConstr(grb.quicksum(to_ship[warehouse, customer] for customer in customers) <= capacity,
                            name='capacity.' + str(warehouse))
            for warehouse, capacity in enumerate(capacities)]

<H3>Solving the Model</H3>

Now we have the building blocks needed to build and solve a transportation model. Let's put it all together.

In [ ]:
import gurobipy as grb
def solve_transportation_model(capacities, demands):
    model = grb.Model()
    warehouses = range(len(capacities))
    customers = range(len(demands))
    to_ship = get_ship_vars(model, warehouses, customers)
    model.update()
    capacity_constrs = get_capacity_constrs(model, to_ship, capacities)
    demand_constrs = get_demand_constrs(model, to_ship, demands)
    model.setObjective(grb.quicksum(ship_costs[warehouse][customer]*to_ship[warehouse, customer]
                                for warehouse in warehouses
                                for customer in customers))
    model.optimize()
    for (warehouse, customer), var in sorted(to_ship.iteritems()):
        if var.X > 1e-4:
            print "Ship", var.X, "units from warehouse", warehouse, "to customer", customer

In [ ]:
solve_transportation_model(capacities, demands)